In [1]:
import numpy as np
import pandas as pd

In [2]:
num_data = pd.read_csv("data.csv")

In [3]:
target = "Class_b\'2\'"
pos = num_data[target].mean()
neg = 1 - pos

# Train-test split

In [4]:
from sklearn.model_selection import train_test_split
import tensorflow as tf

from keras import models
from keras import layers
from keras import optimizers 
from keras import metrics

In [5]:
def sample(data, cut, target):
    assert(cut > 0)
    return data.groupby(target, group_keys=False).apply(lambda x: x.sample(frac=1/cut))    

In [6]:
def get_train_test_split(data, target, cut, random_state = None):
    num_data_new = sample(data, cut, target)
    
    train, test = train_test_split(num_data_new, test_size=0.2, random_state=random_state) 

    y_train = np.array(train[target])
    y_test = np.array(test[target])

    x_train = np.array(train.drop(columns=[target]))
    x_test = np.array(test.drop(columns=[target]))
    
    return x_train, x_test, y_train, y_test 

# Creating models

In [7]:
params = [15, 32, 32, 1]

In [8]:
input_size = len(num_data.columns) - 1

In [9]:
input_size

14

In [10]:
def make_model(input_size, parameters):
    model = models.Sequential()

    model.add(layers.Dense(parameters[0], activation='relu', input_shape=(input_size,))) # , kernel_regularizer="l2"
    for i in range(1, len(parameters)-1):
        model.add(layers.Dense(parameters[i], activation="relu")) #, kernel_regularizer="l1")
    model.add(layers.Dense(parameters[-1], activation='sigmoid')) # , kernel_regularizer="l1"
    
    model.compile(optimizer=optimizers.Adam(), 
              loss='binary_crossentropy', 
              metrics=["accuracy"]
              )
    
    return model

In [11]:
volume = len(num_data)

In [12]:
# cuts = [1, 2, 4, 8, 16, 32, 64, 128, 254, 1024, 2048]
cuts = [1, 2, 4, 8, 16]

In [13]:
print("Cuts are {}".format(cuts))
print("Volume of data is {}".format(list(map(lambda x: int(volume / x), cuts))))

Cuts are [1, 2, 4, 8, 16]
Volume of data is [14980, 7490, 3745, 1872, 936]


In [14]:
get_model_directory = lambda series, cut: "models/{}_series/{}_cut/".format(series, cut)
get_model_name = lambda number:  "m_{}.h5".format(number)

# Logging

In [15]:
import datetime 
def log_preamble(log_file_name):
    global tries, epochs, series, file_name
    time_stamp = datetime.datetime.now()

    with open(log_file_name, "a") as log_file:
        log_file.write(f"\n\nExecuted on time is {datetime.datetime.now()}\n")
        log_file.write(f"Tries: {tries}, epochs: {epochs}, series = {series}\n")
        log_file.write(f"Network configuration is {params}\n")
    return time_stamp

def log_final(log_file_name, time_stamp):
    time_stamp_new = datetime.datetime.now()
    with open(log_file_name, "a") as log_file:
        log_file.write(f"Finished successfully at {time_stamp_new}\n")
        log_file.write(f"Total time = {(time_stamp_new - time_stamp).total_seconds()}\n")

In [16]:
model_name = lambda series_num, cut_val, model_num: get_model_directory(series_num, cut_val) + get_model_name(model_num)

# Training models

In [17]:
from keras.callbacks import ModelCheckpoint

def model_train(data_split, batch_size, epochs, 
                random_target=False, discard = True, model = None, init_epoch = None):
    if not discard:
        model = make_model(input_size, params)
    elif model is None:
        print("Must supply a model if discard is set to False")
        return None # 
     
    if not discard: 
        early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor='val_loss', 
        verbose=1,
        patience=15,
        mode='auto',
        restore_best_weights=True)

        callbacks = [early_stopping]
    else:
        callbacks = []
        
    x_train, x_test, y_train, y_test = data_split

    if random_target:
        y_train = np.random.randint(2, size = len(y_train))
        y_test = np.random.randint(2, size = len(y_test))  

    if model is not None: 
        batch_size = len(x_train)

    history = model.fit(x_train, y_train,
                    epochs=epochs,
                    initial_epoch= init_epoch,
                    batch_size=min(batch_size, len(x_train)),  
                    validation_data=(x_test, y_test), 
                    callbacks = callbacks) 
    return history, model

In [19]:
series = [10503]
tries = 60 
epochs = None
batch_size = None
histories = [[[] for _ in cuts] for _ in series] 

log_file = "model_log.txt"
t = log_preamble(log_file)

model = make_model(input_size, params)

for i, s in enumerate(series):
    for cut_number, cut in enumerate(cuts):
        split = get_train_test_split(num_data, target, cut)
        for number in range(tries):
            model_name_current = model_name(s, cut, number)
            history, model_to_save = model_train(split, None, number+1, discard = False, model = model,
                                                 init_epoch = number) 
            
            model_to_save.save(model_name_current)
            histories[i][cut_number].append(history)

log_final(log_file, t)

1/1 [==============================] - 1s 1s/step - loss: 0.6979 - accuracy: 0.4791 - val_loss: 0.7030 - val_accuracy: 0.4756
Epoch 2/2
1/1 [==============================] - 1s 887ms/step - loss: 0.6980 - accuracy: 0.4557 - val_loss: 0.7134 - val_accuracy: 0.4579
Epoch 3/3
1/1 [==============================] - 1s 949ms/step - loss: 0.7007 - accuracy: 0.4429 - val_loss: 0.6981 - val_accuracy: 0.4579
Epoch 4/4
1/1 [==============================] - 1s 907ms/step - loss: 0.6899 - accuracy: 0.5527 - val_loss: 0.6882 - val_accuracy: 0.5447
Epoch 5/5
1/1 [==============================] - 2s 2s/step - loss: 0.7326 - accuracy: 0.4471 - val_loss: 0.7471 - val_accuracy: 0.4539
Epoch 6/6
1/1 [==============================] - 3s 3s/step - loss: 0.6911 - accuracy: 0.5420 - val_loss: 0.6850 - val_accuracy: 0.5711
Epoch 7/7
1/1 [==============================] - 3s 3s/step - loss: 0.6935 - accuracy: 0.5070 - val_loss: 0.6945 - val_accuracy: 0.5117
Epoch 8/8
1/1 [==============================] - 